In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
filedir = 'data/analysis_crime_seoul'

In [ ]:
filenames = os.listdir(filedir)

In [ ]:
csv_list =[]
for file in filenames:
    if file.endswith('.csv'):
        csv_list.append(file)
file = csv_list[-1]
_crime_anal_police=pd.read_csv("{}/{}".format(filedir, file), encoding ='cp949')
_crime_anal_police.head()

In [ ]:
police_station = _crime_anal_police['구분'].unique()
police_station

In [ ]:
crime_name = _crime_anal_police['죄종'].unique()
crime_name

In [ ]:
crime_class= _crime_anal_police['발생검거'].unique()
crime_class

In [ ]:
result_list = []

for p in police_station:
    temp_dict ={
        '구분': p
    }
    for n in crime_name:
        for c in crime_class:
            key = n+c
            con1 =_crime_anal_police['구분']== p
            con2 =_crime_anal_police['죄종']== n
            con3 =_crime_anal_police['발생검거']==c

            value = _crime_anal_police[con1 & con2 & con3]['건수'].values[0]
            temp_dict[key] = value
    result_list.append(temp_dict)
result_list

In [ ]:
crime_anal_police = pd.DataFrame(result_list)
crime_anal_police

In [ ]:
crime_anal_police['구분'] = crime_anal_police['구분'].apply(lambda x:"서울"+x+"경찰서")
crime_anal_police

In [ ]:
from pprint import pprint
import requests


station_address=[] #주소
station_lng=[] #경도(x)
station_lat=[] #위도(y)

# search 버전 크롤링
url ='https://map.naver.com/v5/api/search'
params={
    'caller':'pcweb'
    
}

for police in crime_anal_police['구분']:
    params['query'] = police
    resp = requests.get(url, params)
    
    data =resp.json()
    geo_info = data['result']['place']['list'][0]
    #roadAdress가 없으면 address를 찾고 이것도없으면 null
    address = geo_info.get('readAddress')
    if not address:
        address = geo_info.get('address')
    
    station_address.append(
        address
    )
    station_lng.append(
        geo_info['x']
    )
    station_lat.append(
        geo_info['y']
    )
station_address, station_lng, station_lat

### Data 저장하기(주소,위도, 경도) p82-ln[7]

In [ ]:
#instantSearch
from time import sleep
import random

url ='https://map.naver.com/v5/api/instantSearch'
params={
    'lang':'ko',
    'caller':'pcweb',
    'types':'place',
    'coords':'37.57553518251637,126.98491573333742'
}
station_address=[] #주소
station_lng=[] #경도(x)
station_lat=[] #위도(y)

for police in crime_anal_police['구분']:
    sleep(random.randint(1,3))
    
    params['q'] = police
    resp = requests.get(url, params)
    
    station_data =resp.json()
    geo_info = station_data['poi'][0]
    #roadAdress가 없으면 address를 찾고 이것도없으면 null
    address = geo_info.get('jibunAddress')
    if not address:
        address = geo_info.get('address')
    
    station_address.append(
        address
    )
    station_lng.append(
        geo_info['x']
    )
    station_lat.append(
        geo_info['y']
    )
station_address, station_lng, station_lat

In [ ]:
zip(station_address, station_lat, station_lng)

In [ ]:
list(zip(station_address, station_lat, station_lng))

In [ ]:
# dict zip은 value값처럼 출력
dict(zip(station_address, station_lat))

### 구 분류하기(p85)

In [ ]:
crime_anal_police.head()

In [ ]:
# #책의 코드
# gu_name=[]

# for name in station_address:
#     tmp=name.split()
    
#     tmp_gu=[gu for gu in tmp if gu[-1] == '구'][0]
    
#     gu_name.append(tmp_gu)
    
# crime_anal_police['구별']=gu_name
# crime_anal_police.head()

#or 다음과 같이도 가능
gu_name=[]
for name in station_address:
    tmp=name.split()
    gu_name.append(tmp[1])
crime_anal_police['구별']=gu_name
crime_anal_police.head()

In [ ]:
crime_anal_police['구별'].value_counts()

#### 형식 동일하게 만들기(구분->관서별)

In [ ]:
crime_anal_police.rename(columns={
    '구분':'관서명'
}, inplace=True)
crime_anal_police.head()

#### 수정한 파일 저장(p87)-ln[14]

In [ ]:
file

In [ ]:
import os

In [ ]:
if not os.path.exists(filedir+'/modified'):
    os.makedirs(filedir+'/modified')

In [ ]:
modified_file= "modified/{}".format(file)

In [ ]:
crime_anal_police.to_csv("{}/{}".format(filedir, modified_file), sep=',', encoding='utf-8')

In [ ]:
crime_anal_police['구별'].value_counts()

### 구별로 정리
### 2-5 Panda-pinot_table

In [ ]:
pd.pivot_table?

In [ ]:
df=crime_anal_police.copy()

In [ ]:
df.head()

In [ ]:
#index를 지정하지 않았다면, 숫자형 데이터만 남음.
pd.pivot_table(df, index=['구별'])

In [ ]:
df[df['구별']=='중구']

In [ ]:
sample= pd.pivot_table(df, index=['구별'])
sample.loc[sample.index=='중구',:]

#### index를 지정하지않으면 숫자형 데이터만 남음.
### index 여러개 지정. -index지정 순서가 pivot의 영향을 끼침.

In [ ]:
pd.pivot_table(df, index=['구별','관서명'])

### value지정 (특정 값만 남기기)

In [ ]:
pd.pivot_table(df, index=['구별','관서명'], values=['절도검거','절도발생'])

### aggfunc 지정하기

In [ ]:
pd.pivot_table(df, index=['구별','관서명'], values=['절도검거','절도발생'],aggfunc=np.max)
#강남구->(강남경찰서, 수서경찰서 max값)
#위와 아래는 동일한 작동을 하지만 내부적 연산 속도에 차이 발생.(numpy 속도 빠름)
#pd.pivot_table(df, index=['구별'], values=['절도검거', '절도발생'], aggfunc=max)

### aggfunc 여러개 지정하기

In [ ]:
#강남구 -> (강남경찰서, 수서경찰서 최대값 및 평균, 최소값)
#Multilndex columns
pd.pivot_table(df, index=['구별','관서명'], values=['절도검거','절도발생'],aggfunc=[np.max, np.mean, np.min])

### piovt_table최종 (+NA값 처리/margin=총계)

In [ ]:
pd.pivot_table(df, index=['구별','관서명'], values=['절도검거','절도발생'],aggfunc=[np.max, np.mean, np.min], fill_value=0)

In [ ]:
#강남구->(강남경찰서, 수서경찰서 합계 및 평균)
pd.pivot_table(df,
              index=['구별'],
              values=['절도검거','절도발생'],
              aggfunc=[np.max, np.mean],
              fill_value=0,
              margins=True,
              margins_name="총계")

### 2-6.Pivot_table을 이용해서 데이터 정리하기

* 예저1.pivot_table을 이용해서 crime_anal_police 데이터의 구별로 데이터를 정리하고 합계를 표시하시오.
그 결과를 crime_anal로 저장하시오

In [ ]:
crime_anal =pd.pivot_table(crime_anal_police, index=['구별'], aggfunc=np.sum)
crime_anal.head()

* 예제2. crime_anal의 각 범죄의 검거와 발생을 가지고 각각[<범죄>검거율]을 입력하시오.
단, 100이 넘어가면 100으로 변경하시오. 그 후 기존[<범죄>검거]데이터는 삭제하시오

In [ ]:
crime_name_list =['강간','강도','살인','절도','폭력']

for crime_name in crime_name_list:
    # 검거율 계산
    temp_series =crime_anal.loc[:, crime_name+"검거"]/crime_anal.loc[:, crime_name+"발생"]*100
    temp_series[temp_series>100]=100
    
    #검거율 df에 추가
    crime_anal[crime_name+"검거율"]=temp_series
    
    #검거 데이터 삭제
    crime_anal.drop(crime_name+"검거", axis=1, inplace=True)
    #아래의 데이터삭제 코드도 가능
    # del crime_anal[crime_name+"검거"]

* 예제3.[<범죄>발생]columns의 "발생"이라는 단어를 제거하시오
(절도발생 ->절도)

In [ ]:
dict(zip([i + "발생" for i in crime_name_list], crime_name_list))

In [ ]:
column_names=dict(zip([i + "발생" for i in crime_name_list], crime_name_list))
column_names

In [ ]:
crime_anal.rename(columns =column_names, inplace=True)

In [ ]:
crime_anal.head()

### 데이터 분포도 보기

In [ ]:
crime_anal.describe()
#데이터의 분포도가 각각 다르다 -> 스케일링 필요 -> 정규화 or 표준화(예-살인과 폭력에 비교)

### 2-7. 데이터 표현으 위해 다듬기

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
col =['강간','강도','살인','절도','폭력']

In [ ]:
x = crime_anal[col].values
min_max_scaler =MinMaxScaler()

#변환기는 fit과 transform 함수 내장. (합친게 fit_transform)
#fit은 변환할 값 저장
#transform은 값 변환
x_scaled = min_max_scaler.fit_transform(x.astype(float))

#오리지널 데이터, 스케일된 데이터
x, x_scaled

In [ ]:
#정규화된 anal_norm data
crime_anal_norm = pd.DataFrame(x_scaled, columns=col, index=crime_anal.index)
crime_anal_norm

In [ ]:
col2 =[i+"검거율" for i in col]
crime_anal_norm[col2] = crime_anal[col2]
crime_anal_norm.head()

* CCTV데이터 가져오기

In [ ]:
result_CCTV =pd.read_csv('./data/CCTV_reslut.csv', encoding='utf-8', index_col='구별')

In [ ]:
result_CCTV

#### 연습문제1.-crime_anal_norm 데이터프레임에 인구수와 CCTV의 소계를 추가하시오.

In [ ]:
crime_anal_norm[['인구수','CCTV']]=result_CCTV[['인구수','소계']]
crime_anal_norm.head()

In [ ]:
# 방법1
#연습문제1
crime_anal_norm[['인구수','CCTV']]=result_CCTV[['인구수','소계']]

* 기존데이터 빼기

In [ ]:
crime_anal_norm.drop(["인구수","CCTV"], axis=1, inplace=True)

In [ ]:
#방법2
#연습문제1
crime_anal_norm.drop(["인구수", "CCTV"], axis=1, inplace=True)
crime_anal_norm =crime_anla_merge(result_CCTV[["소계","인구수"]], on='구별')
crime_anal_norm.rename(columns={
    "소계":"CCTV"
}, inplace=True)
crime_anal_norm

#### 연습문제2-범죄 발생건수의 합을, '범죄'라는 column으로 두어 저장하고, 범죄검거건수의 합을 "검거"라는 column으로 저장하라.

In [ ]:
col

In [ ]:
col2

In [ ]:
crime_anal_norm['범죄']=np.sum(crime_anal_norm[col], axis=1)
crime_anal_norm['검거']=np.sum(crime_anal_norm[col2], axis=1)
crime_anal_norm

### 2-7 좀 더 편리한 시각화 도구-Seaborn

* seaborn을 설치합니다

* seaborn은 내부적으로 matplotlib사용 ->plot을 그리려면 import되어 있어야함

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns

In [ ]:
x=np.linspace(0,14,100)
y1=np.sin(x)
y2 = 2*np.sin(x+0.5)
y3 = 3*np.sin(x+1.0)
y4 = 4*np.sin(x+1.5)

plt.figure(figsize=(6,4))
plt.plot(x, y1, x.y2, x.y3, x.y4)

plt.show()

#### seaborn 자체 데이터로 연습하기

In [ ]:
#tips 데이터-요일별 점심, 저녁, 흡연 여부와 식사금액과 팁을 정리한 테이터
tips=sns.load_dataset('tips')
tips.head(5)

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x="day", y="total_bill", data=tips)
plt.show()

#### hue라는 인자로 구분하기

In [ ]:
plt.figure(figsize=(8,6))
#흡연 여부로 분류하여 플로팅 --> 흡연자가 금액의 범위가 대체로 크다.
#hue에 범주형 데이터를 주면, 범주에 따라 다르게 시각화 함.
sns.boxplot(x='day',y='total_bill', data=tips, hue='smoker', palette='Set3')

#### Implot으로 선형 그래프 그리기

### Import -->scatter처럼 데이터 분포, 직선으로 regression결과 + 유효범위 잡아줌

In [ ]:
sns.set_style("darkgrid")
sns.lmplot(x="total_bill", y="tip", data=tips, size=7)
plt.show()

In [ ]:
# palette -->색상 지정
sns.lmplot(x="total_bill", y="tip", hue="smoker", data=tips, palette="Set1", size=7)
plt.show()

### 비행기 데이터

In [ ]:
flights= sns.load_dataset('flights')
flights.head()

In [ ]:
# index, columns, value
flights =flights.pivot("month", "year", "passengers")
flights.head()

* seaborn heatmap - 데이터 경향 파악

In [ ]:
plt.figure(figsize=(10,8))
# annot: 개수 표현 여부
sns.heatmap(flights, annot=True, fmt='d')
#얼마나 많은지 분포도 파악:fmt:정수(d):annot:숫자표시여부

#### lris 데이터

In [ ]:
sns.set(style='ticks')
iris = sns.load_dataset('iris')
iris.head()

#### pairplot그리기

In [ ]:
sns.pairplot(iris)

 ### hue파라미터 구분하기

In [ ]:
sns.pairplot(iris, hue="species")
#범주형 데이터로 설계

#### variable 줘서 그리기

In [ ]:
sns.pairplot(iris, vars=['sepal_length', 'sepal_width', 'petal_width'])

#### regressio 함께 그리기

In [ ]:
#sns.pairplot(iris, kind='reg', size=3)
sns.pairplot(iris, kind='reg', height=2)

#### x와 y의 변수 다르게 줘서 pairplot그리기

In [ ]:
sns.pairplot(iris,
            kind='reg',
            x_vars=['sepal_width', 'petal_width'],
            y_vars=['sepal_length', 'petal_length'],
            hue='species',
            height=3)

### 2-8. 범조 데이터 시각화 하기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# 한글입력
import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']=False

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path ="C:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

### 연습문제1 -crime_anal_norm을 [강도,살인, 폭력]에 대한 상관관계를 pairplot을 이용해 분석하시오. (regression 그래프와 함께 그리시오)

In [ ]:
sns.pairplot(crime_anal_norm, 
             kind='reg',
             vars=['강도','살인','폭력'])

### 연습문제2. pairplot을 x 변수는['인구수',cctv'], y변수는['살인검거율','폭력검거율']로 하고 pairplot을 regression 그래프와 함께 그리시오

In [ ]:
sns.pairplot(crime_anal_norm,
            kind='reg',   #그래서 선형
            x_vars=['인구수', 'CCTV'],   #x축 데이터값
            y_vars=['살인검거율', '폭력검거율'],   #y축 데이터값
            height=3   #그래프 크기
            )

In [ ]:
# 검거율의 합께를 100으로 한정 짓기. ->정규화 일종.
tmp_max = crime_anal_norm['검거'].max()
crime_anal_norm['검거']=crime_anal_norm['검거']/tmp_max *100

# 정렬
crime_anal_norm_sort = crime_anal_norm.sort_values(by="검거", ascending=False)
#crime_anal_norm_sort

In [ ]:
target_col =col2

crime_anal_norm_sort = crime_anal_norm.sort_values(by="검거",  ascending=False)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(crime_anal_norm_sort[target_col], annot=True, fmt="f", linewidths=.5)   # annot은 숫자
plt.title("범죄 검거 비율 (정규화된 검거의 합)")

#### 연습문제 -범죄비율을 히트맵으로 표현하시오

In [ ]:
target_col =col.copy() + ['범죄']

crime_anal_norm['범죄']=crime_anal_norm['범죄']/5 #5개를 더해 만들었으니 스케일링

crime_anal_norm_sort=crime_anal_norm.sort_values(by="범죄", ascending=False)
plt.figure(figsize=(10,10))
sns.heatmap(crime_anal_norm_sort[target_col], annot=True, fmt="f", linewidths=.5)
plt.title("범죄 비율 (정규화된 발생 건수)")

### 2-9 지도 시가화 도구 Folium

In [ ]:
try:
    import folium
except ModuleNotFoundError:
    !pip install folium
    import folium

In [ ]:
map_osm =folium.Map(location=[45.5236, -122.6750], zoom_start=10, tiles="Stamen Toner")
map_osm

#### folium(Map, Marker, Circle, CircleMarker) +add_to(map)

In [ ]:
map2 =folium.Map(location=[37.5502,126.982], tiles="Stamen Toner", zoom_start=13)
folium.Marker([37.5572,126.982], popup='The waterfront').add_to(map2)

folium.Circle([37.5522,126932], radius=70, popup='Laurelfurst Park').add_to(map2)
folium.CircleMarker([37.5402,126.882], radius=40, popup='Laurelfurst Park', color="#3186cc",
                   fill_color='#3186cc').add_to(map2)
map2

In [ ]:
import folium
import pandas as pd
state_unemployment = './data/02. folium_US_Unemployment_Oct2012.csv'

state_data =pd.read_csv(state_unemployment)
state_data.head()

In [ ]:
state_geo='./data/02. folium_us-states.json'

In [ ]:
import json
with open(state_geo, 'r') as f:
    data=f.read()

In [ ]:
data

In [ ]:
d=json.loads(data)
d

* choropleth

In [ ]:
map= folium.Map(location=[40,-98], zoom_start=4)
map.choropleth(geo_data=state_geo, data = state_data,
              columns=['State', 'Unemployment'],
              key_on='feature.id',
              fill_color='YlGn',
              legend_name='Unemployment Rate(%)')
map

In [ ]:
import json
geo_path ='./data/02. skorea_municipalities_geo_simple.json'
geo_str =json.load(open(geo_path, encoding='utf-8'))

In [ ]:
crime_anal_raw=crime_anal_police.copy()

In [ ]:
crime_anal_raw.head(1)

In [ ]:
#서별 검거울, 구별 범죄 발생율 구한 뒤 데이터 두개를 Map에 중첩표현
crime_anal_raw['lat'] =station_lat
crime_anal_raw['lng'] =station_lng

col =['살인검거','강도검거','강간검거','절도검거','폭력검거']
tmp =crime_anal_raw[col]/crime_anal_raw[col].max()

crime_anal_raw['검거']=np.sum(tmp, axis=1)
crime_anal_raw.head()

In [ ]:
map=folium.Map(location=[37.5502, 126982], zoom_start=11)

for n in crime_anal_raw.index:
    folium.Marker([crime_anal_raw['lat'][n],
                  crime_anal_raw['lng'][n]]).add_to(map)
map

In [ ]:
crime_anal.plot.hist()

In [ ]:
crime_anal.hist()